## Machine Learning Exercise 3: Bias and Variance

**Bias** refers to the error introduced by approximating a complex real-world problem with a simplified model, while **variance** refers to the model's sensitivity to fluctuations in the training data. A linear regression model has high bias and low variance; it makes strong assumptions about the data (linearity) but is stable across different datasets. If these strong assumptions are not correct, there will be places where it systematically overestimates or underestimates. In contrast, a decision tree model has low bias and high variance; it can capture complex patterns but is prone to overfitting, especially if deep and unpruned. This means that it can start to memorize the training data rather than capturing patterns that generalize.

### 1. Fit a linear regression model to the housing data, using sqft_living to predict price. Check the mean squared error on the training data and the test data.

In [124]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression, RidgeCV, LassoCV
from sklearn.metrics import confusion_matrix, RocCurveDisplay, classification_report, classification_report, mean_squared_error, root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, make_scorer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor

In [4]:
house_sales_df = pd.read_csv("data/kc_house_data.csv")
house_sales_df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [5]:
X = house_sales_df[["sqft_living"]]
y = house_sales_df["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)
reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
print("mse training data {}:".format(mse))

mse training data 65713942542.233665:


**Predictions based on X_train tell you how well your model fits the data it has already seen during training.**

**A model that performs very well on X_train (with low errors, like MSE) could indicate that it has successfully learned patterns in the training data. However, if the accuracy is too high, it might suggest overfitting, where the model memorizes the training data instead of learning to generalize.**

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)
reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("mse testing data {}:".format(mse))

mse testing data 74509993356.49603:


**Predictions based on X_test allow you to assess how well your model generalizes to unseen data—real-world performance.**

**If the performance on X_test is significantly worse than on X_train, it indicates overfitting—the model struggles to generalize beyond the training data.**

**Conversely, if the error is high for both X_train and X_test, the model might be underfitting—it doesn't capture the underlying patterns effectively.**

**Conclusion: The difference between the errors is notable but not excessively large. Typically, overfitting is characterized by a very low training error combined with a significantly higher testing error, showing that the model fits the training data well but struggles with unseen data.
Here, while the testing error is slightly higher, the gap between the training and testing MSE isn't drastic enough to immediately conclude overfitting. Instead, it suggests potential underfitting or general model inefficiency, as both errors are quite high.**

### 2. Repeat this but with a [DecisionTreeRegresor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html). Again check the mean squared error on the training data and the test data. How does what you see differ from the linear regression model?

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)
tree_test = DecisionTreeRegressor(random_state = 42)
tree_test.fit(X_train, y_train)
tree_test.predict(X_test)
y_pred_test = tree_test.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred_test)
print("mse testing data {}:".format(mse_test))

mse testing data 79044952597.92511:


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)
tree_train = DecisionTreeRegressor(random_state = 42)
tree_train.fit(X_train, y_train)
y_pred_train = tree_train.predict(X_train)
mse_train = mean_squared_error(y_train, y_pred_train)
print("mse training data {}:".format(mse_train))

mse training data 48410745078.45624:


**In comparison to the earlier Linear Regression model, this suggests a larger gap between the training and testing errors in the Decision Tree model. This larger gap is a red flag for potential overfitting. Decision Trees, being highly flexible models, tend to overfit the training data if not properly constrained.**

### One way of avoiding overfitting is by restricting the flexibility of the model. We can do this with a decision tree by restricting the number of splits that it can perform. 

### 3. Fit a DecisionTreeRegressor where you restrict the max_depth to 5. Again check the mean squared error on the training data and the test data. What do you notice now?

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)
tree = DecisionTreeRegressor(random_state = 42, max_depth = 5)
tree.fit(X_train, y_train)
tree.predict(X_test)
y_pred = tree.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("mse testing data {}:".format(mse))

mse testing data 73264250436.88351:


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)
tree = DecisionTreeRegressor(random_state = 42, max_depth = 5)
tree.fit(X_train, y_train)
tree.predict(X_train)
y_pred = tree.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
print("mse training data {}:".format(mse))

mse training data 56030892751.02288:


**Conclusion: By adding max_depth=5, ive constrained the Decision Tree to prevent it from becoming overly complex, which likely reduced overfitting.**


**Root Node:**

The starting point of the tree, where the data is initially split based on the feature (input variable) that minimizes the mean squared error (MSE) for regression tasks.


**Internal Nodes** (Decision Nodes):

These are points in the tree where further splitting occurs. Each node contains:

A feature on which the split is based.

A threshold value or condition.

Statistical measures to determine the "quality" of the split, like the reduction in variance or mean squared error for regression.

**Leaf Nodes** (Terminal Nodes):

These are the endpoints of the tree and represent the final predictions.

In regression, each leaf node contains the predicted target value, typically the mean or median of the target values for the subset of data that reaches that leaf.


Root: sqft_living <= 3000?
   ├── Yes: sqft_living <= 1500?
   │      ├── Yes: Price = $250,000
   │      ├── No: sqft_living <= 2000?
   │             ├── Yes: Price = $400,000
   │             ├── No: Price = $500,000
   ├── No: sqft_living <= 4000?
          ├── Yes: Price = $700,000
          ├── No: sqft_living <= 5000?
                 ├── Yes: Price = $850,000
                 ├── No: Price = $1,000,000

### When working with machine learning models, we often have to balance bias and variance. This is called the [bias-variance tradeoff](https://en.wikipedia.org/wiki/Bias%E2%80%93variance_tradeoff). One method of this is through [regularization](https://www.ibm.com/think/topics/regularization), where we restrict the complexity of the model, adding some bias but reducing the variance, which can lead to a lower mean squared error on the test set.

### Lasso and ridge regression do this by adding a penalty term based on the size of the coefficients. Smaller coefficients means that the model has less flexibility. The neat thing about these types of models is that they determine how to allocate the coefficients automatically as part of the model fitting process, so we can start with a large set of potential predictors and allow the model fitting to determine which ones to focus on.

### For the next part of the exercise, we'll see how we can add complexity to our model but control the complexity through regularization.

### 4. So far, we've only been predicting based off of the square footage of living space. Fit a new linear regression model using all variables besides id, date, price, and zipcode. How well does this model perform on the test set compared to the model with just square footage of living space?

In [20]:
X = house_sales_df.drop(["id", "date", "price", "zipcode"], axis=1)
y = house_sales_df['price'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)
reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("mse testing data {}:".format(mse))

mse testing data 44093475276.572525:


**Conclusion: The MSE with multiple predictor's is significantly less than the MSE of the linear regression model with only the sq_ft for the predictor**

### 5. Try fitting a lasso and ridge model. Becuase lasso and ridge have penalty terms based on the size of the coefficients, and the size of the coefficients depends on the scale of the variable, you'll want to scale the features first so that they are on comparable scales. Create a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) object where the first step is applying a [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) and the second step is either a lasso or ridge model. Because these models have a hyperparameter controlling regularization strength, you'll want to use the [LassoCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html) and [RidgeCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html) models, which will select values for the regularization strength using cross-validation.

#### LassoCV and RidgeCV are specialized regression models in Scikit-learn that automatically select the best hyperparameter (𝛼) for Lasso Regression and Ridge Regression, respectively, using cross-validation.
    * Lasso CV uses L1 regularization, which penalizes the absolute values of coefficients. It encourages sparsity, meaning it can shrink some coefficients to zero, effectively performing feature selection.
    * Ridge CV uses L2 regularization, which penalizes the square of coefficients. It shrinks coefficients but does not eliminate them, making it useful when all predictors are relevant, but multicollinearity exists (predictors are highly correlated).
    * Both models simplify the hyperparameter tuning process because they have built-in cross-validation, so you don’t need to use tools like GridSearchCV separately.
#### StandardScaler 
    * Standardizing:  The combination of centering and scaling. Specifically, it transforms data such that each feature has a mean of 0 and a standard deviation of 1. This ensures features contribute equally to the model.

In [77]:
X = house_sales_df.drop(["id", "date", "price", "zipcode"], axis=1)
y = house_sales_df['price'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)
steps = [("StandardScaling", StandardScaler()), 
         ("RidgeRegression", RidgeCV())]
pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
r2 = pipeline.score(X_test, y_test)
mse = mean_squared_error(y_test, y_pred)
print("R^2 Ridge Regression score {}:".format(r2))
print("MSE Ridge Regression score {}:".format(mse))

R^2 Ridge Regression score 0.6945638061327064:
MSE Ridge Regression score 44094763932.52483:


In [46]:
X = house_sales_df.drop(["id", "date", "price", "zipcode"], axis=1)
y = house_sales_df['price'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)
steps = [("StandardScaling", StandardScaler()), 
         ("LassoRegression", LassoCV())]
pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
r2 = pipeline.score(X_test, y_test)
mse = mean_squared_error(y_test, y_pred)
print("R^2 Lasso Regression score {}:".format(r2))
print("Lasso MSE Regression score {}:".format(mse))

R^2 Lasso Regression score 0.6943271474783299:
Lasso MSE Regression score 44128929521.62934:


**Interpretation: model explains roughly 69% of the variance in the target variable. The slight edge for Ridge Regression might suggest it fits your data a bit better, potentially due to its L2 regularization handling multicollinearity more effectively than the L1 regularization of Lasso.**

### You likely didn't see much difference between the regular linear regression model and the lasso or ridge model. Let's see what happens if we add more complexity through feature interactions. We can capture more complex relationships between the predictor variables and the target variable by multiplying the predictors together before fitting the model. For example, the interaction between sqft_living and bedrooms will let the model capture if the impact of square footage depends on the number of bedrooms.

### 6. Add [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) to your pipeline after the standard scaler. Try using degree 2 features. How does this change the performance of the regular linear regression model, the lasso model, and the ridge model? 

#### PolynomialFeatures is a powerful preprocessing tool in Scikit-learn that creates polynomial features from your existing input data (features). In essence, it generates a new feature matrix containing not only the original features but also their polynomial combinations and interactions up to a specified degree.
    * Suppose you specify a polynomial degree of 2. PolynomialFeatures will create a new feature matrix containing:
        * The original features
        * The squared terms of each feature
        * The interaction terms between pairs of features

In [81]:
X = house_sales_df.drop(["id", "date", "price", "zipcode"], axis=1)
y = house_sales_df['price'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)

steps = [("StandardScaling", StandardScaler()),
         ("PolynomialFeatures", PolynomialFeatures(degree=2)),
         ("LinearRegression", LinearRegression())]
pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE Linear Regression score {}:".format(mse))

MSE Linear Regression score 30926921716.856598:


In [83]:
X = house_sales_df.drop(["id", "date", "price", "zipcode"], axis=1)
y = house_sales_df['price'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)

steps = [("StandardScaling", StandardScaler()),
         ("PolynomialFeatures", PolynomialFeatures(degree=2)),
         ("RidgeRegression", RidgeCV())]
pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE Ridge Regression score {}:".format(mse))

MSE Ridge Regression score 30933833753.383926:


In [86]:
X = house_sales_df.drop(["id", "date", "price", "zipcode"], axis=1)
y = house_sales_df['price'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)

steps = [("StandardScaling", StandardScaler()),
         ("PolynomialFeatures", PolynomialFeatures(degree=2)),
         ("LassoRegression", LassoCV())]
pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE Lasso Regression score {}:".format(mse))

MSE Lasso Regression score 30414023904.74287:


**Interpretation: Adding in the PolynomialFeature function drove down the MSE a lot.**

### The lasso penalty tends to cause some coeffients to zero out, so it can be viewed as a method of automatic feature selection.
### 7. Look at the set of coefficients for the lasso model. What percentage of the coefficients are zero? What are the largest non-zero coefficients?

In [100]:
X = house_sales_df.drop(["id", "date", "price", "zipcode"], axis=1)
y = house_sales_df['price'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)

steps = [("StandardScaling", StandardScaler()),
         ("PolynomialFeatures", PolynomialFeatures(degree=2)),
         ("LassoRegression", LassoCV())]
pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)

Lasso_model = pipeline.named_steps["LassoRegression"]
Lasso_coef = Lasso_model.coef_
print("MSE Lasso coefficients {}:".format(Lasso_coef))

MSE Lasso coefficients [ 0.00000000e+00 -0.00000000e+00  1.29631038e+04  9.30898037e+04
 -0.00000000e+00 -7.40640157e+03  0.00000000e+00  0.00000000e+00
  2.55901284e+04  9.49584662e+04  3.48881548e+03  0.00000000e+00
 -1.89758098e+04  0.00000000e+00  6.91512974e+04 -2.23709363e+04
  4.64911392e+04 -0.00000000e+00  1.34675819e+02 -3.42301923e+03
 -5.54586218e+03  0.00000000e+00  0.00000000e+00 -8.21509899e+02
 -0.00000000e+00  2.68731640e+02  4.54886920e+02 -0.00000000e+00
 -7.66847183e+02 -4.16932354e+02 -2.08144768e+03 -0.00000000e+00
  5.53855599e+03  2.71218569e+02  1.37786566e+03 -0.00000000e+00
  1.94692202e+04 -3.77436715e+03 -5.05144532e+03 -0.00000000e+00
  2.70220671e+03 -1.27452732e+03  9.42420011e+02  7.47394194e+03
  0.00000000e+00  3.62574286e+03 -4.04970904e+03  0.00000000e+00
 -1.84220237e+03 -6.94222056e+02  0.00000000e+00  4.36724451e+03
 -5.07516407e+03 -0.00000000e+00  6.50325965e+03  0.00000000e+00
  0.00000000e+00  3.34707747e+04  0.00000000e+00  0.00000000e+00
  

In [118]:
np.sum(Lasso_coef)

341509.3047541333

In [116]:
CoefLessThanZero = np.sum(Lasso_coef < 0) / np.sum(Lasso_coef)
print("The number of coefficients less than zero {}:".format(CoefLessThanZero))

The number of coefficients less than zero 0.00014348071726852986:


In [114]:
np.sort(Lasso_coef)[::-1]

array([ 9.49584662e+04,  9.30898037e+04,  6.91512974e+04,  4.64911392e+04,
        3.34707747e+04,  2.55901284e+04,  1.98399910e+04,  1.95939812e+04,
        1.94692202e+04,  1.88427149e+04,  1.38986201e+04,  1.32212070e+04,
        1.29631038e+04,  1.12849069e+04,  1.11287126e+04,  1.03483293e+04,
        8.01702457e+03,  7.50998142e+03,  7.47394194e+03,  7.39133793e+03,
        7.17101323e+03,  6.50325965e+03,  6.44427138e+03,  5.82835092e+03,
        5.53855599e+03,  5.52222147e+03,  4.86254680e+03,  4.44911272e+03,
        4.36724451e+03,  4.24076570e+03,  3.62574286e+03,  3.59010509e+03,
        3.56115240e+03,  3.48881548e+03,  3.45395611e+03,  3.34830745e+03,
        2.95522228e+03,  2.87635097e+03,  2.70220671e+03,  2.45260297e+03,
        2.22387716e+03,  1.90564838e+03,  1.82774005e+03,  1.64917083e+03,
        1.43351210e+03,  1.38982217e+03,  1.37786566e+03,  1.34178964e+03,
        1.21118164e+03,  1.13592527e+03,  1.05481548e+03,  9.42420011e+02,
        8.91210628e+02,  

**Interpretation: In Lasso Regression, a coefficient of zero means that the corresponding feature is completely excluded from the model.**

### 8. A new hyperparameter that we have is the degree of the polynomial we're using. So that we're not overfitting to the test set, we need to use cross-validation to select this value. Set up a [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to try out polynomial degrees from 1 to 3 and to try LinearRegression, LassoCV, and RidgeCV models. Use 'neg_mean_squared_error' as the error_score. Which combination does it find does the best? 

In [196]:
X = house_sales_df.drop(["id", "date", "price", "zipcode"], axis=1)
y = house_sales_df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)


pipeline = Pipeline([
    ("standardscaling", StandardScaler()),
    ("polynomialfeatures", PolynomialFeatures()),
    ("model", LinearRegression()) 
])


param_grid = {
    "polynomialfeatures__degree": [1, 2, 3],
    "model": [LinearRegression(), RidgeCV(), LassoCV(cv=5, max_iter=10000)] 
}


grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring="neg_mean_squared_error", 
    cv=5
)

grid_search.fit(X_train, y_train)

# Get the best combination of parameters and the corresponding score
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Use the best model to make predictions on the test set
print("Best parameters:", best_params)
print("Best neg MSE score:", grid_search.best_score_)

# Evaluate the test set performance
y_pred = best_estimator.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)

print(f"Test Mean Squared Error: {test_mse}")

Best parameters: {'model': LinearRegression(), 'polynomialfeatures__degree': 2}
Best neg MSE score: -25869569633.99911
Test Mean Squared Error: 30926921716.856598
